<a href="https://colab.research.google.com/github/NKASG/Traffic_police_gesture/blob/main/frame_picture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os

# Replace 'your_folder_path' with the actual path to the folder containing video and CSV files
folder_path = '/content/drive/MyDrive/PoliceGestureLong/train'

# Output folder for saving frames based on assigned numbers
output_folder = '/content/drive/MyDrive/PoliceGestureLong'
os.makedirs(output_folder, exist_ok=True)

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    base_name, extension = os.path.splitext(filename)

    if extension == ".mp4":
        # Form the full path for the video file
        video_path = os.path.join(folder_path, filename)

        # Form the full path for the corresponding CSV file
        csv_filename = f"{base_name}.csv"
        csv_file_path = os.path.join(folder_path, csv_filename)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Get the frames per second (fps) of the video
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Calculate the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Initialize an empty list to store frames
        frames_list = []

        # Read and save each frame to the list
        for frame_number in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break  # Break the loop if there are no more frames

            # Append each frame to the list
            frames_list.append(frame)

        # Release the video capture object
        cap.release()

        # Check if the CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file_path, header=None)

            # Check if the number of columns in the CSV file matches the number of frames
            if len(df.columns) == len(frames_list):
                # Iterate over unique assigned numbers
                unique_assigned_numbers = df.iloc[0].unique()
                for assigned_number in unique_assigned_numbers:
                    # Filter frames corresponding to the current assigned number
                    filtered_frames = [frame for frame, number in zip(frames_list, df.iloc[0]) if number == assigned_number]

                    # Create a folder for the assigned number
                    output_folder_assigned = os.path.join(output_folder, f'assigned_{assigned_number}')
                    os.makedirs(output_folder_assigned, exist_ok=True)

                    # Save each frame in the folder
                    for idx, frame in enumerate(filtered_frames):
                        output_file_path = os.path.join(output_folder_assigned, f'{base_name}_frame_{idx + 1}.png')
                        cv2.imwrite(output_file_path, frame)

            else:
                print(f"For {base_name}, Number of columns in the CSV file does not match the number of frames.")
        else:
            print(f"CSV file not found for {base_name}.")
